In [1]:
import pandas as pd
from pymongo import MongoClient

In [2]:
date_after = pd.to_datetime("1/1/2007  00:00:00 AM")
date_before = pd.to_datetime("1/1/2023  00:00:00 AM")

In [3]:
ROOTPATH = "/Users/nuhash/Desktop/Code/Thesis/"

In [4]:
client = MongoClient('mongodb://localhost:27017/')
db = client['test2']

In [5]:
crimeKeys = ['PdId', 'IncidntNum', 'Incident Code','datetime', 'Category', 'Descript', 'PdDistrict', 'Resolution', 'location']
def crimeArrayToObject(crime):
    return [
        str(crime[0]), str(crime[1]), str(crime[2]), str(crime[3]), str(crime[4]), str(crime[5]), str(crime[6]), str(crime[7]) , str(crime[8])]

In [6]:
def mapDatasetByDate(dataset):
    mapByDate = {}
    allData = dataset.values
    
    for i in allData:
        date = str(i[3])

        if date in mapByDate:
            mapByDate[date]["crimes"].append(crimeArrayToObject(i))    
        else:
            mapByDate[date] = {
                "date": date,
                "crimeKey": crimeKeys,
                "crimes": [crimeArrayToObject(i)]
            }
    return mapByDate

# chicago crime data trimming

In [ ]:
PATH_DA = ROOTPATH+"dataset\\processed\\chicago_data_2007_2023.csv"

dataset = pd.read_csv(ROOTPATH+'dataset\\Crimes_-_2001_to_Present.csv')

print(dataset.shape , dataset.columns)

In [ ]:
# convert date to datetime
dataset['datetime'] = pd.to_datetime(dataset['Date'], format='%m/%d/%Y %I:%M:%S %p')
dataset = dataset.sort_values(by='datetime')
dataset = dataset[(dataset.datetime)>=date_after]
dataset = dataset[(dataset.datetime)<date_before]
dataset['PdId'] = dataset["FBI Code"]
dataset['IncidntNum'] = dataset.ID
dataset['Incident Code'] = dataset['IUCR']
dataset['Category'] = dataset['Primary Type']
dataset['Descript'] = dataset['Description']
dataset['PdDistrict'] = dataset['District']
dataset['Resolution'] = dataset['Arrest']
dataset['location'] = dataset['Location']
dataset = dataset[['PdId', 'IncidntNum', 'Incident Code', 'datetime', 'Category', 'Descript', 
                   'PdDistrict', 'Resolution', 'location']]
print(dataset.shape)

In [ ]:
documentData = mapDatasetByDate(dataset)

In [ ]:
finalData = []
for date in documentData:
    finalData.append(documentData[date])

In [ ]:
len(finalData)

In [ ]:
collection = db['chicago.crime.filtered']
# Convert DataFrame records to dictionary format for MongoDB insertion
collection.insert_many(finalData)

In [ ]:
del dataset
del documentData
del finalData

In [ ]:
import gc 
print(gc.collect())

# SF CRIME data trimming

In [ ]:
PATH_SF_PROCESSED = ROOTPATH+"dataset\\processed\\sf_data_2007_2023.csv"

dataset_new = pd.read_csv(ROOTPATH+'dataset\\Police_Department_Incident_Reports__2018_to_Present.csv')
dataset_old = pd.read_csv(ROOTPATH+'dataset\\Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv')

print(dataset_new.columns, dataset_new.size)
print(dataset_old.columns, dataset_old.size)

In [ ]:
print(dataset_new['Resolution'].unique(),
dataset_old['Resolution'].unique())

In [ ]:
date_after = pd.to_datetime("1/1/2007  00:00:00 AM")
date_before = pd.to_datetime("1/1/2023  00:00:00 AM")
# 2023/03/13 11:41:00 PM
dataset_new['datetime'] = pd.to_datetime(dataset_new['Incident Datetime'], format='%Y/%m/%d %I:%M:%S %p')
# convert resolution to boolean
dataset_new['Resolution'] = dataset_new['Resolution'].apply(lambda x: True if (x == 'Cite or Arrest Adult' or x=='Exceptional Adult' or x=='Arrest') else False)
dataset_new = dataset_new.sort_values(by = ['datetime'])

dataset_new['PdId'] = dataset_new['Row ID']
dataset_new['IncidntNum'] = dataset_new['Incident Number']
dataset_new['Category'] = dataset_new['Incident Category']
dataset_new['Descript'] = dataset_new['Incident Description']
dataset_new['PdDistrict'] = dataset_new['Police District']
dataset_new['Resolution'] = dataset_new['Resolution']
dataset_new['location'] = dataset_new['Point']

dataset_new = dataset_new[crimeKeys]

In [ ]:
arrested = ['ARREST, BOOKED', 'ARREST, CITED',
            'NOT PROSECUTED',
            'PROSECUTED BY OUTSIDE AGENCY',
            'DISTRICT ATTORNEY REFUSES TO PROSECUTE',
            'COMPLAINANT REFUSES TO PROSECUTE',
            'EXCEPTIONAL CLEARANCE', 'PROSECUTED FOR LESSER OFFENSE']

def isArrest(x):
    if x in arrested:
        return True
    else:
        return False
    
dataset_old['datetime'] = pd.to_datetime(dataset_old['Date']+' '+dataset_old['Time'], format='%m/%d/%Y %H:%M')
dataset_old = dataset_old.sort_values(by = ['datetime'])
# convert resolution to boolean
dataset_old['Resolution'] = dataset_old['Resolution'].apply(lambda x: isArrest(x))
dataset_old = dataset_old[crimeKeys]


In [ ]:
df = pd.concat([dataset_old, dataset_new], ignore_index=True)
print(df.shape)
df = df[(df.datetime)>=date_after]
df = df[(df.datetime)<date_before]
df = df.sort_values(by = ['datetime'])

In [ ]:

documentData = mapDatasetByDate(df)
finalData = []
for date in documentData:
    finalData.append(documentData[date])

print(len(finalData))



In [ ]:
collection = db['sf.crime.filtered']
collection.insert_many(finalData)

In [ ]:
del df
del dataset_old
del dataset_new
del documentData
del finalData

# Philly Crime trim

In [9]:
PATH_PHILLY_PROCESSED = ROOTPATH+"dataset\\processed\\philly_data_2007_2023.csv"

dataset = pd.read_csv(ROOTPATH+'dataset/philly_2006_2023.csv')

/var/folders/jq/f31p5js52f51prlhxvhddy2w0000gn/T/ipykernel_12013/361677048.py:3: DtypeWarning: Columns (15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(ROOTPATH+'dataset/philly_2006_2023.csv')


In [10]:
print(dataset.shape , dataset.columns)
dataset.sample(5)

(3175805, 18) Index(['objectid', 'dc_dist', 'psa', 'dispatch_date_time', 'dispatch_date',
       'dispatch_time', 'hour', 'dc_key', 'location_block', 'ucr_general',
       'text_general_code', 'point_x', 'point_y', 'lat', 'lng', 'the_geom',
       'cartodb_id', 'the_geom_webmercator'],
      dtype='object')


,objectid,dc_dist,psa,dispatch_date_time,dispatch_date,dispatch_time,hour,dc_key,location_block,ucr_general,text_general_code,point_x,point_y,lat,lng,the_geom,cartodb_id,the_geom_webmercator
784762,1548035,3,K,2009-09-09 12:10:00,2009-09-09,08:10:00,8.0,2.009030e+11,1100 BLOCK ELLSWORTH ST,1400,Vandalism/Criminal Mischief,-75.163323,39.936124,39.936124,-75.163323,NaN,NaN,NaN
286299,1934599,6,I,2007-04-25 01:45:00,2007-04-24,21:45:00,21.0,2.007060e+11,900 BLOCK MARKET ST,1400,Vandalism/Criminal Mischief,-75.155162,39.951334,39.951334,-75.155162,NaN,NaN,NaN
941968,2182632,26,2,2010-07-09 17:50:00,2010-07-09,13:50:00,13.0,2.010260e+11,1900 BLOCK N 05TH ST,1400,Vandalism/Criminal Mischief,-75.142549,39.981354,39.981354,-75.142549,NaN,NaN,NaN
993633,2184528,22,4,2010-10-07 18:22:00,2010-10-07,14:22:00,14.0,2.010221e+11,1200 BLOCK N HOLLYWOOD ST,1800,Narcotic / Drug Law Violations,-75.183925,39.975357,39.975357,-75.183925,NaN,NaN,NaN
1928293,103081,12,4,2015-11-10 01:16:00,2015-11-09,20:16:00,20.0,2.015121e+11,5100 BLOCK REGENT ST D,300,Robbery No Firearm,-75.218403,39.941867,39.941867,-75.218403,NaN,NaN,NaN


In [12]:

# convert date to datetime
# 2010-10-21 05:14:00+00
dataset['datetime'] = pd.to_datetime(dataset['dispatch_date_time'])
# remove timezone
dataset['datetime'] = dataset['datetime'].dt.tz_localize(None)
print(dataset['datetime'].sample(5))


1629165   2014-03-27 05:52:00
326582    2007-06-29 14:10:00
1496878   2013-06-18 01:42:00
2444609   2019-02-19 20:05:00
2340874   2018-06-19 00:03:00
Name: datetime, dtype: datetime64[ns]


In [13]:

dataset = dataset.sort_values(by='datetime')

date_after = pd.to_datetime("2007-01-01 00:00:00")
date_before = pd.to_datetime("2023-01-01 00:00:00")

dataset = dataset[(dataset.datetime)>=date_after]
dataset = dataset[(dataset.datetime)<date_before]

dataset['PdId'] = dataset["dc_dist"]
dataset['IncidntNum'] = dataset["objectid"]
dataset['Incident Code'] = dataset['ucr_general']
dataset['Category'] = dataset['text_general_code']
dataset['Descript'] = dataset['text_general_code']
dataset['PdDistrict'] = dataset['dc_dist']
dataset['location'] = dataset['point_x'].astype(str) + ',' + dataset['point_y'].astype(str)
dataset['Resolution'] = dataset['dispatch_date_time'].apply(lambda x: True)

dataset = dataset[crimeKeys]

print(dataset.shape)


(2783525, 9)


In [14]:
documentData = mapDatasetByDate(dataset)
finalData = []
for date in documentData:
    finalData.append(documentData[date])

print(len(finalData))


2224543


In [15]:
collection = db['philly.crime.filtered']

collection.insert_many(finalData)

In [ ]:
del dataset
del documentData
del finalData

In [ ]:
queryDate = pd.to_datetime("1/1/2010")

#Find crimes after 2010 in chicago, sf and philly
chicago = db['chicago.crime.filtered']
sf = db['sf.crime.filtered']
philly = db['philly.crime.filtered']

# convert date to mongodb date 
# check if date >= queryDate and date < queryDate + 1
query = {
    "date": {
        "$gte": str(queryDate),
        "$lt": str(queryDate + pd.DateOffset(days=1))
    }
}
print(query)

chicagoData = chicago.find(query)
sfData = sf.find(query)
phillyData = philly.find(query)


In [ ]:
crimeKeys

In [ ]:
#create a dataframe from the cursor
data = []
for i in chicagoData:
    data.extend(i['crimes'])

print(len(data))
for i in sfData:
    data.extend(i['crimes'])
print(len(data))
for i in phillyData:
    data.extend(i['crimes'])
print(len(data))

df = pd.DataFrame(data, columns=crimeKeys)
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values(by='datetime')
print(df.shape)

In [ ]:
df['datetime'].unique()